In [1]:
import pandas as pd
import numpy as np
import math
import os
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt 
import seaborn as sns            # visualization tool
import matplotlib.cm as cm       #

In [2]:
def zero_crossing(temp,threshold):
    zc = 0
    for i in range(len(temp)-1):
        if((temp[i]*temp[i+1]) < 0 and abs(temp[i]-temp[i+1]) >= threshold):
            zc += 1
    return zc

In [3]:
def process_one_csv(file_name,k):
    data = pd.read_csv('/home/amaya/emp/Realtime-Sign-Language-Translation-to-Speech-DNN/all/User'+str(k)+'/'+file_name)
    global classes
    
    class_ = file_name[0:len(file_name)-14]
    
    # print(data.head(2))

    columns = list(data.columns)[1:]
    # features = ['MAV','RMS','VAR','SSI','MAX','MIN']
    
    vector = []
    # global out_columns
    
    for item in columns:
        temp = list(data[item])
        # calculating MAV ***
        # out_columns.append(item+features[0])
        abs_val = list(map(abs,temp))
        vector.append(np.mean(abs_val))

        # calculating RMS ***
        # out_columns.append(item+features[1])
        vector.append(np.sqrt(np.mean(np.array(temp)**2)))

        # calculate variance ***
        # out_columns.append(item+features[2])
        x_mean = np.mean(temp)
        dif = temp - x_mean
        vector.append(np.mean(np.array(dif)**2))

        # calculating ssi ***
        # out_columns.append(item+features[3])
        vector.append(np.sum(np.array(temp)**2))

        # calculating max ***
        # out_columns.append(item+features[4])
        vector.append(max(temp))

        # calculating min ***
        # out_columns.append(item+features[5])
        vector.append(min(temp))
        
        # calculating zero crossing
        if((item[0] == 'E') or (item[0] == 'G')):
            vector.append(zero_crossing(temp,1))
        elif(item[0] == 'A'):
            vector.append(zero_crossing(temp,0.15))
        
    if class_ in classes:
        vector.append(classes.index(class_))
    else:
        classes.append(class_)
        vector.append(classes.index(class_))
        
    return vector



In [4]:
all_features = ['EMG0LMAV', 'EMG0LRMS', 'EMG0LVAR', 'EMG0LSSI', 'EMG0LMAX', 'EMG0LMIN','EMG0ZC', 'EMG1LMAV', 'EMG1LRMS', 'EMG1LVAR', 'EMG1LSSI', 'EMG1LMAX', 'EMG1LMIN','EMG1ZC', 'EMG2LMAV', 'EMG2LRMS', 'EMG2LVAR', 'EMG2LSSI', 'EMG2LMAX', 'EMG2LMIN', 'EMG2ZC','EMG3LMAV', 'EMG3LRMS', 'EMG3LVAR', 'EMG3LSSI', 'EMG3LMAX', 'EMG3LMIN', 'EMG3ZC','EMG4LMAV', 'EMG4LRMS', 'EMG4LVAR', 'EMG4LSSI', 'EMG4LMAX', 'EMG4LMIN', 'EMG4ZC','EMG5LMAV', 'EMG5LRMS', 'EMG5LVAR', 'EMG5LSSI', 'EMG5LMAX', 'EMG5LMIN', 'EMG5ZC','EMG6LMAV','EMG6LRMS', 'EMG6LVAR', 'EMG6LSSI', 'EMG6LMAX', 'EMG6LMIN', 'EMG6ZC','EMG7LMAV', 'EMG7LRMS', 'EMG7LVAR', 'EMG7LSSI', 'EMG7LMAX', 'EMG7LMIN', 'EMG7ZC','AXLMAV', 'AXLRMS', 'AXLVAR', 'AXLSSI', 'AXLMAX', 'AXLMIN', 'AXLZC','AYLMAV', 'AYLRMS', 'AYLVAR', 'AYLSSI', 'AYLMAX', 'AYLMIN', 'AYLZC','AZLMAV', 'AZLRMS', 'AZLVAR', 'AZLSSI', 'AZLMAX', 'AZLMIN', 'AZLZC','GXLMAV', 'GXLRMS', 'GXLVAR', 'GXLSSI', 'GXLMAX', 'GXLMIN', 'GXLZC','GYLMAV', 'GYLRMS', 'GYLVAR', 'GYLSSI', 'GYLMAX', 'GYLMIN', 'GYLZC','GZLMAV', 'GZLRMS', 'GZLVAR', 'GZLSSI', 'GZLMAX', 'GZLMIN', 'GZLZC','ORLMAV', 'ORLRMS', 'ORLVAR', 'ORLSSI', 'ORLMAX', 'ORLMIN', 'OPLMAV', 'OPLRMS', 'OPLVAR', 'OPLSSI', 'OPLMAX', 'OPLMIN', 'OYLMAV', 'OYLRMS', 'OYLVAR', 'OYLSSI', 'OYLMAX', 'OYLMIN', 'EMG0RMAV', 'EMG0RRMS', 'EMG0RVAR', 'EMG0RSSI', 'EMG0RMAX', 'EMG0RMIN', 'EMG0RZC', 'EMG1RMAV', 'EMG1RRMS', 'EMG1RVAR', 'EMG1RSSI', 'EMG1RMAX', 'EMG1RMIN','EMG1RZC', 'EMG2RMAV', 'EMG2RRMS', 'EMG2RVAR', 'EMG2RSSI', 'EMG2RMAX', 'EMG2RMIN','EMG2RZC', 'EMG3RMAV', 'EMG3RRMS', 'EMG3RVAR', 'EMG3RSSI', 'EMG3RMAX', 'EMG3RMIN','EMG3RZC', 'EMG4RMAV', 'EMG4RRMS', 'EMG4RVAR', 'EMG4RSSI', 'EMG4RMAX', 'EMG4RMIN','EMG4RZC', 'EMG5RMAV', 'EMG5RRMS', 'EMG5RVAR', 'EMG5RSSI', 'EMG5RMAX', 'EMG5RMIN','EMG5RZC', 'EMG6RMAV', 'EMG6RRMS', 'EMG6RVAR', 'EMG6RSSI', 'EMG6RMAX', 'EMG6RMIN','EMG6RZC', 'EMG7RMAV', 'EMG7RRMS', 'EMG7RVAR', 'EMG7RSSI', 'EMG7RMAX', 'EMG7RMIN','EMG7RZC', 'AXRMAV', 'AXRRMS', 'AXRVAR', 'AXRSSI', 'AXRMAX', 'AXRMIN','AXRZC', 'AYRMAV', 'AYRRMS', 'AYRVAR', 'AYRSSI', 'AYRMAX', 'AYRMIN','AYRZC', 'AZRMAV', 'AZRRMS', 'AZRVAR', 'AZRSSI', 'AZRMAX', 'AZRMIN','AZRZC', 'GXRMAV', 'GXRRMS', 'GXRVAR', 'GXRSSI', 'GXRMAX', 'GXRMIN','GXRZC', 'GYRMAV', 'GYRRMS', 'GYRVAR', 'GYRSSI', 'GYRMAX', 'GYRMIN','GYRZC', 'GZRMAV', 'GZRRMS', 'GZRVAR', 'GZRSSI', 'GZRMAX', 'GZRMIN','GZRZC', 'ORRMAV', 'ORRRMS', 'ORRVAR', 'ORRSSI', 'ORRMAX', 'ORRMIN', 'OPRMAV', 'OPRRMS', 'OPRVAR', 'OPRSSI', 'OPRMAX', 'OPRMIN', 'OYRMAV', 'OYRRMS', 'OYRVAR', 'OYRSSI', 'OYRMAX', 'OYRMIN','CLASS']
# 233 columns
selected_features = ['ORLMAX', 'OPLMAV', 'OPLRMS', 'OPLVAR', 'OPLSSI', 'OPLMIN', 'OYLMAV','OYLVAR', 'OYLMIN', 'EMG1RVAR', 'EMG1RSSI', 'EMG2RVAR', 'EMG3RMAV','EMG3RVAR', 'EMG6RMAV', 'EMG6RRMS', 'EMG7RMAV', 'AXRMAV', 'AXRRMS','AXRVAR', 'AXRSSI', 'AXRMAX', 'AYRMAV', 'AYRVAR', 'AZRMAV', 'AZRRMS','AZRVAR', 'AZRSSI', 'AZRMIN', 'GXRMAV', 'GXRRMS', 'GXRVAR', 'GXRSSI','GZRMAV', 'GZRMAX', 'GZRMIN', 'ORRMAV', 'ORRVAR', 'ORRMIN', 'OPRMAV','OPRRMS', 'OPRVAR', 'OPRSSI', 'OPRMIN', 'OYRVAR', 'OYRSSI']
# 46 features selected from random forest
ocolumns = ['ORLMAX', 'OPLMAV', 'OPLRMS', 'OPLVAR', 'OPLSSI', 'OPLMIN', 'OYLMAV','OYLVAR', 'OYLMIN', 'EMG1RVAR', 'EMG1RSSI', 'EMG2RVAR', 'EMG3RMAV','EMG3RVAR', 'EMG6RMAV', 'EMG6RRMS', 'EMG7RMAV', 'AXRMAV', 'AXRRMS','AXRVAR', 'AXRSSI', 'AXRMAX', 'AYRMAV', 'AYRVAR', 'AZRMAV', 'AZRRMS','AZRVAR', 'AZRSSI', 'AZRMIN', 'GXRMAV', 'GXRRMS', 'GXRVAR', 'GXRSSI','GZRMAV', 'GZRMAX', 'GZRMIN', 'ORRMAV', 'ORRVAR', 'ORRMIN', 'OPRMAV','OPRRMS', 'OPRVAR', 'OPRSSI', 'OPRMIN', 'OYRVAR', 'OYRSSI', 'CLASS']
# 47 columns
output = []
classes=[]
# print(len(out_columns))

# process_one_csv(files[0])
for k in range(1,11):
    if(k!=5):
        files = os.listdir('/home/amaya/emp/Realtime-Sign-Language-Translation-to-Speech-DNN/all/User'+str(k)+'/')
    else:
        continue
    for file in files:
        if(file[-1] == 'v'):      # to avoid _DS_STORE files and any other hidden files
            output.append(process_one_csv(file,k))
            
initial_df = pd.DataFrame(output,columns = all_features)
processed = initial_df[ocolumns]
processed.shape

(550, 47)

In [5]:
# feature scaling of data
scaler = preprocessing.MinMaxScaler()   # since the data set is not gaussian
scaled_df = scaler.fit_transform(processed[ocolumns[:len(ocolumns)-1]])
X = pd.DataFrame(scaled_df, columns = ocolumns[:len(ocolumns)-1])
y = processed['CLASS']

In [6]:
# splitting the dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.20)

In [7]:
# setting up the layers
model = keras.Sequential([
    keras.layers.Dense(46,input_dim = 46,activation = 'relu'),
    keras.layers.Dense(230,activation = 'relu'), # 450 for 232 features
    keras.layers.Dense(20,activation = 'softmax')
])

# compile the model
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

W0824 04:00:28.890142 140225153967872 deprecation.py:506] From /home/amaya/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
# Training the model
model.fit(X_train,y_train,epochs = 250)

Epoch 1/250
440/440 [==============================] - 0s 269us/sample - loss: 2.9845 - acc: 0.0432
Epoch 2/250
440/440 [==============================] - 0s 60us/sample - loss: 2.8969 - acc: 0.1386
Epoch 3/250
440/440 [==============================] - 0s 46us/sample - loss: 2.8091 - acc: 0.2409
Epoch 4/250
440/440 [==============================] - 0s 46us/sample - loss: 2.7021 - acc: 0.2477
Epoch 5/250
440/440 [==============================] - 0s 44us/sample - loss: 2.5438 - acc: 0.3136
Epoch 6/250
440/440 [==============================] - 0s 44us/sample - loss: 2.3920 - acc: 0.3659
Epoch 7/250
440/440 [==============================] - 0s 50us/sample - loss: 2.2369 - acc: 0.4205
Epoch 8/250
440/440 [==============================] - 0s 45us/sample - loss: 2.1057 - acc: 0.4114
Epoch 9/250
440/440 [==============================] - 0s 47us/sample - loss: 1.9839 - acc: 0.4568
Epoch 10/250
440/440 [==============================] - 0s 48us/sample - loss: 1.8738 - acc: 0.4932
Epoch 11

In [9]:
# evaluate accuracy
test_loss,test_acc = model.evaluate(X_test,y_test)
print(test_acc)

110/110 [==============================] - 0s 280us/sample - loss: 0.4732 - acc: 0.8818
0.8818182


In [10]:
# predict
predictions = model.predict(X_test)
for i in range(20):
    print(np.argmax(predictions[i]))

17
12
7
9
11
6
2
18
18
19
17
3
17
10
4
7
7
19
10
3


In [11]:
for i in range(20):
    print(list(y_test)[i])

4
12
7
9
9
6
2
12
18
19
17
14
17
10
4
7
7
19
10
8
